<a href="https://colab.research.google.com/github/doomer-doomer/Unsloth-x-Generative-Model/blob/main/PPT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

[NEW] Llama-3.1 8b, 70b & 405b are trained on a crazy 15 trillion tokens with 128K long context lengths!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.5: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    #r=16 initially
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.5 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
from datasets import load_dataset

# PPT format template with generalized titles for at least 5 slides per row
ppt_template = """\
Slide 1: Introduction
-------------------
Title: {title}

Introduction:
{introduction}

-------------------

Slide 2: Agenda
-------------------
Key Points:
{key_points}

-------------------

Slide 3: Key Concepts
-------------------
Detailed Summary:
{detailed_summary}

-------------------

Slide 4: Proposed Solution
-------------------
Insights:
{additional_insights}

-------------------

Slide 5: Conclusion
-------------------
Conclusion:
{conclusion}

-------------------

Slide 6: Recommendations
-------------------
Recommendations:
{recommendations}

-------------------

Slide 7: Questions?
-------------------
Please feel free to ask any questions!
"""

PROMPT_TEMPLATE = """
Generate a PowerPoint-like presentation response for the following topic:

Title: {title}

Response (in PowerPoint format):
"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    titles = examples["Title"]
    key_points = examples["short_summary"]
    detailed_summaries = examples["Summary"]
    texts = []

    for title, key_point, detailed_summary in zip(titles, key_points, detailed_summaries):
        # Handle NoneType detailed_summary
        if detailed_summary is None:
            detailed_summary = ""  # Set default value if None
        if key_point is None:
            key_point = ""  # Set default value if None

        # Split the detailed summary into two parts for additional insights and conclusion
        mid_index = len(detailed_summary) // 2
        additional_insights = detailed_summary[:mid_index]  # First half for Slide 4
        conclusion = detailed_summary[mid_index:]  # Second half for Slide 5

        # Optional: Add custom recommendations or use the key points if available
        recommendations = key_point if len(key_point) > 0 else "Refer to the insights and analysis for more information."

        # Format each set of slides using the template
        slides_text = ppt_template.format(
            title=title if title else "No Title Available",  # Handle None or missing title
            introduction=key_point[:100] + "..." if key_point else "No Introduction Available",  # Truncated intro from short_summary
            key_points=key_point if key_point else "No Key Points Available",
            detailed_summary=detailed_summary if detailed_summary else "No Detailed Summary Available",
            additional_insights=additional_insights if additional_insights else "No Additional Insights Available",
            conclusion=conclusion if conclusion else "No Conclusion Available",
            recommendations=recommendations
        )

        # Combine PROMPT_TEMPLATE with the slides_text
        full_prompt = PROMPT_TEMPLATE.format(title=title) + slides_text + EOS_TOKEN

        texts.append(full_prompt)

    return {"text": texts}


data_files = {
    "train": "/content/train.csv",
    "test": "/content/test.csv"
}

datasets = load_dataset("csv", data_files="/content/train.csv", split="train")
dataset = datasets.map(formatting_prompts_func, batched = True,)

# # Access the train and test datasets separately
# train_dataset = datasets["train"]
# test_dataset = datasets["test"]

# # Apply the formatting function to create the 5-slide text format for the train dataset
# train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

# # Optional: Apply the same formatting for the test dataset if needed for evaluation
# test_dataset = test_dataset.map(formatting_prompts_func, batched=True)

# Now the 'dataset' contains the formatted text for at least 5 slides per row with generalized titles


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
alpaca_prompt="""
Below is a technical question or task related to Operating Systems. Provide a detailed, accurate, and well-structured response following the guidelines.

### Instruction: {}

### Context:
- Specific Topic Area
- Complexity Level: Intermediate
- Response Type: Explaination

### Input: {}

### Constraints:
- Focus on practical system-level concepts
- Include relevant technical terminology
- Provide examples where applicable
- Reference standard OS mechanisms and algorithms
- Maintain accuracy of technical details

### Required Response Elements:
1. Core Concept Explanation
2. Technical Details and Mechanisms
3. Practical Examples or Use Cases
4. Related OS Components (if applicable)
5. Common Issues or Considerations
6. Best Practices or Optimization Tips

### Response Format:
1. Brief Overview
   - Main concept definition
   - Key points summary

2. Technical Details
   - Implementation specifics
   - Relevant algorithms
   - System calls or APIs

3. Examples
   - Code snippets (if applicable)
   - Real-world scenarios
   - Implementation cases

4. Related Concepts
   - Dependencies
   - Interactions with other OS components

5. Considerations
   - Performance implications
   - Security aspects
   - Common pitfalls

### Response: {}
"""

In [ ]:
import pandas as pd

# Load the CSV files
df_x = pd.read_csv('/content/train.csv')
df_y = pd.read_csv('/content/test.csv')
df_z = pd.read_csv('/content/val.csv')

# Concatenate the dataframes
df_w = pd.concat([df_x, df_y, df_z], ignore_index=True)

# Save the merged dataframe to a new CSV file
df_w.to_csv('w.csv', index=False)

print("Merged CSV saved as 'w.csv'.")

Merged CSV saved as 'w.csv'.


In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["short_summary"]
    inputs       = examples["Title"]
    outputs      = examples["Summary"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("csv",data_files="/content/w.csv", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1399 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    #eval_dataset = test_dataset, ##added later
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #num_train_epochs = 2, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1399 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
8.588 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,399 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 57,016,320


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.993500
2,1.996400
3,1.995600
4,1.870300
5,1.587200
6,1.448500
7,1.309300
8,0.947700
9,0.919500
10,1.010400


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1414.786 seconds used for training.
23.58 minutes used for training.
Peak reserved memory = 12.83 GB.
Peak reserved memory for training = 4.242 GB.
Peak reserved memory % of max memory = 86.995 %.
Peak reserved memory for training % of max memory = 28.763 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

input_question = "List the types of deadlocks and explain each."

# Prepare the input for the model using only the question
input_prompt = f"Question: {input_question}\n\nGenerate a full presentation covering the following sections:\n1. Introduction\n2. Key Points\n3. Detailed Summary\n4. Insights\n5. Conclusion\n6. Recommendations"

# Tokenize the input question for the model
inputs = tokenizer(
    [input_prompt],  # We only provide the question or instruction here
    return_tensors="pt"
).to("cuda")  # Use GPU for faster inference

# Generate the output using the model
outputs = model.generate(
    **inputs,
    max_new_tokens=1500,  # Generate enough tokens for multiple slides
    use_cache=True
)

# Decode the model output to get the generated text
generated_ppt_content = tokenizer.batch_decode(outputs)

In [ ]:
print(generated_ppt_content[0])

<|begin_of_text|>Question: List the types of deadlocks and explain each.

Generate a full presentation covering the following sections:
1. Introduction
2. Key Points
3. Detailed Summary
4. Insights
5. Conclusion
6. Recommendations

Introduction:
Quote:
List items:
- Circular Wait
- Hold and Wait
- No Preemption
- Mutual Exclusion
- Non-Sharable Resource
- Resource Deadlock
- Resource Allocation Graph Deadlock
- Priority Inversion Deadlock
- Deadlock Avoidance
- Deadlock Detection
- Deadlock Recovery
- Deadlock Prevention

Key Points:
List items:
- Circular Wait
- Hold and Wait
- No Preemption
- Mutual Exclusion
- Non-Sharable Resource
- Resource Deadlock
- Resource Allocation Graph Deadlock
- Priority Inversion Deadlock
- Deadlock Avoidance
- Deadlock Detection
- Deadlock Recovery
- Deadlock Prevention

Detailed Summary:
List items:
- Circular Wait
- Hold and Wait
- No Preemption
- Mutual Exclusion
- Non-Sharable Resource
- Resource Deadlock
- Resource Allocation Graph Deadlock
- Prior

In [ ]:
# alpaca_prompt = Copied from above
PROMPT = """
Generate a PowerPoint-like presentation response for the following topic:

{}

Response (in PowerPoint format):

{}
"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    PROMPT.format(
        "Explain Deadlock in Operating System", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1500, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>\nGenerate a PowerPoint-like presentation response for the following topic:\n\nExplain Deadlock in Operating System\n\nResponse (in PowerPoint format):\n\n\nSlide 1: Introduction\n-------------------\nTitle: Deadlock in Operating System\n\nIntroduction:\nList items:\n- A deadlock is a situation in which a set of processes are blocked indefinitely because...\n\n-------------------\n\nSlide 2: Agenda\n-------------------\nKey Points:\nList items:\n- A deadlock is a situation in which a set of processes are blocked indefinitely because each process holds a resource and waits for another resource acquired by some other process.\n\n-------------------\n\nSlide 3: Key Concepts\n-------------------\nDetailed Summary:\nList items:\n- A deadlock is a situation in which a set of processes are blocked indefinitely because each process holds a resource and waits for another resource acquired by some other process.\n\n-------------------\n\nSlide 4: Proposed Solution\n-----------

In [ ]:
print(tokenizer.batch_decode(outputs))

['<|begin_of_text|>\nGenerate a PowerPoint-like presentation response for the following topic:\n\nExplain Deadlock in Operating System\n\nResponse (in PowerPoint format):\n\n\nSlide 1: Introduction\n-------------------\nTitle: Deadlock in Operating System\n\nIntroduction:\nList items:\n- A deadlock is a situation in which a set of processes are blocked indefinitely because...\n\n-------------------\n\nSlide 2: Agenda\n-------------------\nKey Points:\nList items:\n- A deadlock is a situation in which a set of processes are blocked indefinitely because each process holds a resource and waits for another resource acquired by some other process.\n\n-------------------\n\nSlide 3: Key Concepts\n-------------------\nDetailed Summary:\nList items:\n- A deadlock is a situation in which a set of processes are blocked indefinitely because each process holds a resource and waits for another resource acquired by some other process.\n\n-------------------\n\nSlide 4: Proposed Solution\n-----------

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
topic = "Types of deadlocks"
title = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Introduction on "+ topic + " in 50 words", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

key_points = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Give key points on " + topic, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

sum = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Long Summary on the " + topic + " in 500 words", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

example = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Give an example related to the " + topic , # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**title, streamer = text_streamer, max_new_tokens = 2048)
__ = model.generate(**key_points, streamer = text_streamer, max_new_tokens = 2048)
___ = model.generate(**sum, streamer = text_streamer, max_new_tokens = 2048)
____ = model.generate(**example, streamer = text_streamer, max_new_tokens = 2048)


<s>
Below is a technical question or task related to Operating Systems. Provide a detailed, accurate, and well-structured response following the guidelines.

### Instruction: 

### Context:
- Specific Topic Area
- Complexity Level: Intermediate 
- Response Type: Explaination

### Input: Introduction on Types of deadlocks in 50 words

### Constraints:
- Focus on practical system-level concepts
- Include relevant technical terminology
- Provide examples where applicable
- Reference standard OS mechanisms and algorithms
- Maintain accuracy of technical details

### Required Response Elements:
1. Core Concept Explanation
2. Technical Details and Mechanisms
3. Practical Examples or Use Cases
4. Related OS Components (if applicable)
5. Common Issues or Considerations
6. Best Practices or Optimization Tips

### Response Format:
1. Brief Overview
   - Main concept definition
   - Key points summary

2. Technical Details
   - Implementation specifics
   - Relevant algorithms
   - System calls o

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("/content/driver/MyDrive/PPT Model/ppt_model_mistral_v1") # Local saving
tokenizer.save_pretrained("/content/driver/MyDrive/PPT Model/ppt_model_mistral_v1")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/driver/MyDrive/PPT Model/ppt_model_mistral_v1/tokenizer_config.json',
 '/content/driver/MyDrive/PPT Model/ppt_model_mistral_v1/special_tokens_map.json',
 '/content/driver/MyDrive/PPT Model/ppt_model_mistral_v1/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/PPT Model/ppt_model_mistral_v1",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

Unsloth 2024.11.5 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(131072, 5120, padding_idx=10)
        (layers): ModuleList(
          (0-39): 40 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
topic = "Operating System"
title = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Introduction on "+ topic + " in 100 words", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

key_points = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Give key points on " + topic, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

sum = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Explain briefly on the " + topic + " in 500 words", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

example = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Give an example related to the " + topic , # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**title, streamer = text_streamer, max_new_tokens = 1500)
__ = model.generate(**key_points, streamer = text_streamer, max_new_tokens = 1500)
___ = model.generate(**sum, streamer = text_streamer, max_new_tokens = 1500)
____ = model.generate(**example, streamer = text_streamer, max_new_tokens = 1500)


<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:


### Input:
Introduction on Operating System in 100 words

### Response:
An operating system is a software program that manages the computer hardware and software resources, and provides a platform for running applications. It is responsible for allocating resources, managing processes, and providing a user interface.</s>
<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:


### Input:
Give key points on Operating System

### Response:
List items:
- Operating System is a program that acts as an interface between the user and computer hardware.
- It is responsible for managing the computer’s resources, such as memory, CPU, and storage.
- It provides a set of services and functions that allow users to interact with the computer and perform various tasks.
- It is responsible for scheduling a

In [ ]:
from google.colab import drive
drive.mount('/content/driver')

Mounted at /content/driver


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>